In [1]:
!pip install pymorphy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 921.7 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 2.4 MB/s eta 0:00:0000:0100:01
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13705 sha256=b6c823af93c76ffd75bd4cbac4e3da2598248aa24408ed76dd9e79a90618e7f5
  Stored in directory: /Users/avnekrasov/Library/Caches/pip/wheels/7c/d7/8d/2156234738063e3d4a39ba77dc677046100e62766b53807189
Successfully built docopt


In [2]:
!pip install pymorphy2-dicts

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 1.7 MB/s eta 0:00:0000:0100:01


In [3]:
!pip install DAWG-Python

In [4]:
import pandas as pd
import re
import pymorphy2
import joblib as pickle

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from sklearn import tree

In [80]:
df = pd.read_csv('lenta-ru-news.csv')

/var/folders/sr/vm0b7w256vv3544jh36np78w0000gn/T/ipykernel_14934/3394781730.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('lenta-ru-news.csv')


In [81]:
df.head()

,url,title,text,topic,tags,date
0,https://lenta.ru/news/1914/09/16/hungarnn/,1914. Русские войска вступили в пределы Венгрии,Бои у Сопоцкина и Друскеник закончились отступ...,Библиотека,Первая мировая,1914/09/16
1,https://lenta.ru/news/1914/09/16/lermontov/,1914. Празднование столетия М.Ю. Лермонтова от...,"Министерство народного просвещения, в виду про...",Библиотека,Первая мировая,1914/09/16
2,https://lenta.ru/news/1914/09/17/nesteroff/,1914. Das ist Nesteroff!,"Штабс-капитан П. Н. Нестеров на днях, увидев в...",Библиотека,Первая мировая,1914/09/17
3,https://lenta.ru/news/1914/09/17/bulldogn/,1914. Бульдог-гонец под Льежем,Фотограф-корреспондент Daily Mirror рассказыва...,Библиотека,Первая мировая,1914/09/17
4,https://lenta.ru/news/1914/09/18/zver/,1914. Под Люблином пойман швабский зверь,"Лица, приехавшие в Варшаву из Люблина, передаю...",Библиотека,Первая мировая,1914/09/18


In [82]:
df.shape

(800975, 6)

In [83]:
df.tags.value_counts()

Все               453762
Политика           40716
Общество           35202
Украина            22523
Происшествия       19825
                   ...  
Мировой опыт           6
Нацпроекты             6
Вооружение             3
Инновации              1
69-я параллель         1
Name: tags, Length: 94, dtype: int64

In [84]:
df = df_specific_type[df_specific_type.topic.isin(['Россия','Мир','Экономика','Спорт','Культура'])].copy()

In [85]:
df.shape

(494804, 6)

In [86]:
df.isna().sum()

url          0
title        0
text         5
topic        0
tags     15328
date         0
dtype: int64

In [87]:
df[df.text.isna()]['title'].values[0]

'Прокуратура не\xa0нашла национальной почвы в\xa0нападении на\xa0индийского студента'

In [88]:
df.dropna(subset=['text'],inplace = True)

In [89]:
df.topic.value_counts()

Россия       160442
Мир          136620
Экономика     79528
Спорт         64413
Культура      53796
Name: topic, dtype: int64

In [90]:
df.topic.value_counts(normalize=True)

Россия       0.324257
Мир          0.276112
Экономика    0.160728
Спорт        0.130180
Культура     0.108723
Name: topic, dtype: float64

In [91]:
df.isna().sum()

url          0
title        0
text         0
topic        0
tags     15328
date         0
dtype: int64

In [92]:
df[df.text.isna()]['title'].values

array([], dtype=object)

In [93]:
df = df[~df.text.isna()]

In [94]:
df.shape

(494799, 6)

In [95]:
RANDOM_STATE = 42

# Обработка текста

In [24]:
text_transformer = TfidfVectorizer()

In [25]:
%%time
text = text_transformer.fit_transform(df['text'])

CPU times: user 56.9 s, sys: 1.95 s, total: 58.9 s
Wall time: 59.2 s


In [26]:
df['text']

5         Как стало известно агентству Ассошиэйтед Пресс...
6         В зале игровых автоматов в третьем ярусе подзе...
7         Япония приняла решение разморозить кредиты Рос...
8         Британцы отмечают сегодня скорбную дату - втор...
9         В понедельник директор департамента внешних св...
                                ...                        
739168    Президент России Владимир Путин, выступая на з...
739169    Президент России Владимир Путин организовал дл...
739171    Протесты движения «желтых жилетов» в Париже сн...
739174    Швейцарский горнолыжник Марк Гизин неудачно пр...
739175    Президент России Владимир Путин рассказал, что...
Name: text, Length: 494799, dtype: object

In [27]:
text.shape

(494799, 821732)

In [28]:
X_train, X_test, y_train, y_test = train_test_split(text, df['topic'], test_size=0.20, random_state=RANDOM_STATE)

In [29]:
X_train.shape

(395839, 821732)

In [30]:
X_test.shape

(98960, 821732)

In [31]:
%%time
clf = tree.DecisionTreeClassifier()

CPU times: user 47 µs, sys: 87 µs, total: 134 µs
Wall time: 142 µs


In [32]:
clf.fit(X_train,y_train)

DecisionTreeClassifier()

In [33]:
pred = clf.predict(X_test)
print('Качество модели по метрике F1', f1_score(y_test,pred,average='weighted')) 

Качество модели по метрике F1 0.791639515266716


# Изменим токенизатор

In [34]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [36]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/avnekrasov/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [37]:
stop_russian = stopwords.words('russian')

In [39]:
text_transformer = TfidfVectorizer(stop_words=stop_russian, 
                                   ngram_range=(1,1), 
                                   lowercase=True, 
                                   max_features=10000 )
text = text_transformer.fit_transform(df['text'])
X_train, X_test, y_train, y_test = train_test_split(text, df['topic'], test_size=0.2, random_state=RANDOM_STATE)

In [40]:
from sklearn.ensemble import GradientBoostingClassifier

In [43]:
X_train.shape

(395839, 10000)

In [44]:
clf2 = GradientBoostingClassifier()
clf2.fit(X_train,y_train)

GradientBoostingClassifier()

In [45]:
print('TRAIN - Качество модели по метрике F1 (токенизатор настроен)',
      f1_score(y_train,clf2.predict(X_train),average='weighted'))

TRAIN - Качество модели по метрике F1 (токенизатор настроен) 0.8560479482126485


In [46]:
%%time
pred2 = clf2.predict(X_test)
print('TEST - Качество модели по метрике F1 (токенизатор настроен)', f1_score(y_test,pred2,average='weighted'))

TEST - Качество модели по метрике F1 (токенизатор настроен) 0.8540586138542992
CPU times: user 1.36 s, sys: 65.7 ms, total: 1.42 s
Wall time: 1.43 s


In [58]:
with open ('transformer.pkl', 'wb') as f:
    pickle.dump(text_transformer, f)

In [59]:
with open ('clf.pkl', 'wb') as f:
    pickle.dump(clf2, f)

In [96]:
df['text'] = df['text'].str.replace('/','')

# приведем слова к первой форме

In [97]:
pattern = r'[^А-Яа-я]+'

In [98]:
list = [re.sub(pattern, ' ', i) for i in ['проверкаyyy weyweywt iweiwue ><<++ 238238 ? 123 тест']]

In [99]:
list

['проверка тест']

In [100]:
# функция удаления пунктуации и цифр
def remove_trash(list): 
    pattern = r'[^А-Яа-я]+'
    try:
      list = [re.sub(pattern, ' ', i) for i in list] 
#       print (list)
    except Exception as e:
      print(e)
    return list

In [101]:
df['text_clean'] = remove_trash(df['text'])

In [102]:
df[['text','text_clean']].head()

,text,text_clean
5,Как стало известно агентству Ассошиэйтед Пресс...,Как стало известно агентству Ассошиэйтед Пресс...
6,В зале игровых автоматов в третьем ярусе подзе...,В зале игровых автоматов в третьем ярусе подзе...
7,Япония приняла решение разморозить кредиты Рос...,Япония приняла решение разморозить кредиты Рос...
8,Британцы отмечают сегодня скорбную дату - втор...,Британцы отмечают сегодня скорбную дату вторую...
9,В понедельник директор департамента внешних св...,В понедельник директор департамента внешних св...


In [103]:
df.isna().sum()

url               0
title             0
text              0
topic             0
tags          15328
date              0
text_clean        0
dtype: int64

In [104]:
df = df[~df.text_clean.isna()].copy()

In [105]:
#токенизатор удалил мусор

In [124]:
text_transformer = TfidfVectorizer(stop_words=stop_russian, 
                                   ngram_range=(1,1), 
                                   lowercase=True, 
                                   max_features=10000 )
text = text_transformer.fit_transform(df['text_clean'])
X_train, X_test, y_train, y_test = train_test_split(text, df['topic'], test_size=0.2, random_state=RANDOM_STATE)

In [125]:
clf3 = tree.DecisionTreeClassifier(max_depth=3)
clf3.fit(X_train,y_train)
pred3 = clf3.predict(X_test)
print('Качество модели по метрике F1 (токенизатор настроен и убран мусор)', f1_score(y_test,pred3,average='weighted'))

Качество модели по метрике F1 (токенизатор настроен и убран мусор) 0.32884454455872786


# попробуем лемматизацию

In [126]:
morph = pymorphy2.MorphAnalyzer()

In [127]:
# функция лемматизации
def lemmatize(row):
    t = []
    text = row['text_clean']
    for word in text.split():
        if len(word)<=1:
            continue
        p = morph.parse(word)[0]
        t.append(p.normal_form)
    return " ".join(t)

In [128]:
df.text

5         Как стало известно агентству Ассошиэйтед Пресс...
6         В зале игровых автоматов в третьем ярусе подзе...
7         Япония приняла решение разморозить кредиты Рос...
8         Британцы отмечают сегодня скорбную дату - втор...
9         В понедельник директор департамента внешних св...
                                ...                        
739168    Президент России Владимир Путин, выступая на з...
739169    Президент России Владимир Путин организовал дл...
739171    Протесты движения «желтых жилетов» в Париже сн...
739174    Швейцарский горнолыжник Марк Гизин неудачно пр...
739175    Президент России Владимир Путин рассказал, что...
Name: text, Length: 494799, dtype: object

In [129]:
t = []
text = 'Минфин предлагает увеличить размер штрафа'
for word in text.split():
    if len(word)<=3:
        continue
    p = morph.parse(word)[0]
    t.append(p.normal_form)

In [130]:
" ".join(t)

'минфин предлагать увеличить размер штраф'

In [131]:
%%time
df['text_clean_normal'] = df.apply(lemmatize,axis=1)

CPU times: user 2h 28min 47s, sys: 16.7 s, total: 2h 29min 4s
Wall time: 3h 59min 40s


In [132]:
# Запись в файл объекта
with open('my_newfile.pkl','wb') as f:
    pickle.dump(df['text'],f)

In [133]:
#Чтение объекта в память
with open ('my_newfile.pkl','rb') as f:
    data_df = pickle.load(f)

In [134]:
# пример записи объекта в файл
with open('df_norma.pkl','wb') as f:
    pickle.dump(df,f)

In [135]:
# пример чтения объекта из файла в оперативную память
with open('df_norma.pkl','rb') as f:
    df = pickle.load(f)

In [136]:
df.head()

,url,title,text,topic,tags,date,text_clean,text_clean_normal
5,https://lenta.ru/news/1999/08/31/stancia_mir/,"Космонавты сомневаются в надежности ""Мира""",Как стало известно агентству Ассошиэйтед Пресс...,Россия,Все,1999/08/31,Как стало известно агентству Ассошиэйтед Пресс...,как стать известно агентство ассошиэйтед пресс...
6,https://lenta.ru/news/1999/08/31/vzriv/,Взрыв в центре Москвы: пострадало 30 человек,В зале игровых автоматов в третьем ярусе подзе...,Россия,Все,1999/08/31,В зале игровых автоматов в третьем ярусе подзе...,зал игровой автомат третий ярус подземный комп...
7,https://lenta.ru/news/1999/08/31/credit_japs/,Япония кредитует Россию на полтора миллиарда д...,Япония приняла решение разморозить кредиты Рос...,Россия,Все,1999/08/31,Япония приняла решение разморозить кредиты Рос...,япония принять решение разморозить кредит росс...
8,https://lenta.ru/news/1999/08/31/diana/,Британцы отмечают двухлетие смерти Дианы,Британцы отмечают сегодня скорбную дату - втор...,Мир,Все,1999/08/31,Британцы отмечают сегодня скорбную дату вторую...,британец отмечать сегодня скорбный дата вторую...
9,https://lenta.ru/news/1999/08/31/mvf/,Отмытые через Bank of NY деньги не имели отнош...,В понедельник директор департамента внешних св...,Россия,Все,1999/08/31,В понедельник директор департамента внешних св...,понедельник директор департамент внешний связь...


In [137]:
df[['text','text_clean_normal']].tail(10)

,text,text_clean_normal
739161,Одиннадцать человек скончались после приема пи...,одиннадцать человек скончаться после приём пищ...
739162,Россия не настроена серьезно на сохранение Дог...,россия не настроить серьёзно на сохранение дог...
739164,Федеральный суд в техасском городе Форт-Уорт о...,федеральный суд техасский город форт уорт объя...
739166,Россия призвала Китай принять участие в перего...,россия призвать китай принять участие перегово...
739167,Главный тренер «Манчестер Юнайтед» Жозе Моурин...,главный тренер манчестер юнайтед жозе моуринья...
739168,"Президент России Владимир Путин, выступая на з...",президент россия владимир путин выступать на з...
739169,Президент России Владимир Путин организовал дл...,президент россия владимир путин организовать д...
739171,Протесты движения «желтых жилетов» в Париже сн...,протест движение жёлтый жилет париж снова прив...
739174,Швейцарский горнолыжник Марк Гизин неудачно пр...,швейцарский горнолыжник марк гизина неудачно п...
739175,"Президент России Владимир Путин рассказал, что...",президент россия владимир путин рассказать что...


# качество модели после лемматизации

In [139]:
text_transformer = TfidfVectorizer(stop_words=stop_russian, 
                                   ngram_range=(1,1), 
                                   lowercase=True, 
                                   max_features=10000)
text_norm = text_transformer.fit_transform(df['text_clean_normal'])
X_train, X_test, y_train, y_test = train_test_split(text_norm, df['topic'], test_size=0.20, random_state=RANDOM_STATE)

In [140]:
text_transformer.transform(['Минфин предлагает увеличить размер'])

<1x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 3 stored elements in Compressed Sparse Row format>

In [141]:
clf_norm = tree.DecisionTreeClassifier()
clf_norm.fit(X_train,y_train)
pred_norm = clf_norm.predict(X_test)

In [142]:
clf_norm.predict(text_transformer.transform(['Минфин предлагает увеличить размер']))

array(['Мир'], dtype=object)

In [143]:
print('Качество модели по метрике F1 после лемматизации', f1_score(y_test,pred_norm,average='weighted'))

Качество модели по метрике F1 после лемматизации 0.8142661691937688


In [144]:
## Сравниваем вектора текстов

In [145]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


textA = text_transformer.transform(['бывший тренер сборная россия владимир плющев']   )                              

test = cosine_similarity(text_norm, textA)

In [146]:
testdf = pd.DataFrame(test)
testdf.columns=['vector']

In [147]:
testdf.sort_values(by='vector',ascending=False).head(5)

,vector
79742,0.573275
316961,0.558445
46511,0.515492
45830,0.511427
288703,0.502407


In [148]:
df[df.index==103437]['text_clean_normal'].values

array(['связь перекладка инженерный коммуникация строительство эстакада для развязка третий транспортный кольцо ленинградский проспект москва октябрь закрываться движение автотранспорт при съезд по развязка с сторона гостиница советский как сообщить информационный агентство интерфакс гибдд столица движение на участок закрываться ориентировочно до август год также отменяться правый поворот ленинградский проспект на улица нижний башиловка при движение из центр направление дмитровский шоссе выезд внутренний сторона третий кольцо на ленинградский проспект при движение с сторона бегов улица направление область ранее сообщаться что ремонт на третий транспортный кольцо москва быть идти течение день по октябрь быть перекрыть выезд внутренний сторона кольцо на ленинский проспект направление область гибдд обратиться автомобилист просьба заранее планировать поездка учёт маршрут объезд'],
      dtype=object)